## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-06-18-40-48 +0000,nypost,Gavin Newsom roasted for bizarre reaction post...,https://nypost.com/2025/12/06/us-news/gavin-ne...
1,2025-12-06-18-39-19 +0000,bbc,Iran arrests marathon organisers over women no...,https://www.bbc.com/news/articles/c3e0z97qn3eo...
2,2025-12-06-18-38-45 +0000,cbc,"Belgium, Germany and Croatia to play group mat...",https://www.cbc.ca/sports/soccer/2026-world-cu...
3,2025-12-06-18-36-52 +0000,nypost,Over 70K Ukrainian women now serving in army a...,https://nypost.com/2025/12/06/world-news/more-...
4,2025-12-06-18-35-57 +0000,nyt,Crayola Magnetic Blocks Are Recalled Over Chok...,https://www.nytimes.com/2025/12/06/business/cr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2419/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
159,trump,39
418,netflix,16
420,warner,15
111,new,14
549,court,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
235,2025-12-05-19-14-25 +0000,latimes,Supreme Court will hear Trump's plan to restri...,https://www.latimes.com/politics/story/2025-12...,89
220,2025-12-05-19-51-16 +0000,cbc,U.S. Supreme Court to decide whether Trump's b...,https://www.cbc.ca/news/world/supreme-court-tr...,80
226,2025-12-05-19-33-39 +0000,nypost,Supreme Court to decide whether or not Trump’s...,https://nypost.com/2025/12/05/us-news/supreme-...,79
177,2025-12-05-22-05-51 +0000,nyt,Supreme Court Agrees to Review Trump Order Res...,https://www.nytimes.com/2025/12/05/us/politics...,78
209,2025-12-05-20-34-00 +0000,wsj,Supreme Court to Decide if Trump Birthright Ci...,https://www.wsj.com/us-news/law/supreme-court-...,77


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
235,89,2025-12-05-19-14-25 +0000,latimes,Supreme Court will hear Trump's plan to restri...,https://www.latimes.com/politics/story/2025-12...
181,61,2025-12-05-21-56-47 +0000,nyt,Netflix to Buy Warner Bros in $83 Billion Deal,https://www.nytimes.com/2025/12/05/business/wa...
11,42,2025-12-06-18-00-00 +0000,wsj,His Grandfather’s Funeral Felt So…Cold. So He ...,https://www.wsj.com/business/entrepreneurship/...
118,39,2025-12-06-02-00-00 +0000,wsj,Takeaways from Robert F. Kennedy Jr.’s vaccine...,https://www.wsj.com/politics/skeptics-in-the-s...
19,36,2025-12-06-17-01-39 +0000,nypost,Family of Texas A&M cheerleader Brianna Aguile...,https://nypost.com/2025/12/06/us-news/family-u...
51,35,2025-12-06-14-31-08 +0000,cbc,Russia unleashes massive drone and missile att...,https://www.cbc.ca/news/world/russia-ukraine-a...
7,34,2025-12-06-18-10-02 +0000,nypost,Deadly Hong Kong fire came after residents wer...,https://nypost.com/2025/12/06/world-news/hong-...
126,33,2025-12-06-01-02-00 +0000,wsj,Two survivors of a Sept. 2 lethal U.S. strike ...,https://www.wsj.com/politics/national-security...
50,32,2025-12-06-14-35-09 +0000,nypost,Stephen Miller calls billion-dollar Minnesota ...,https://nypost.com/2025/12/06/us-news/stephen-...
2,29,2025-12-06-18-38-45 +0000,cbc,"Belgium, Germany and Croatia to play group mat...",https://www.cbc.ca/sports/soccer/2026-world-cu...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
